Snippet from notebooks...

In [ ]:
# state_fp = df_census_pop_raw['GEOID'].str.slice(9,11).rename('State FIPS').astype(int)
# county_fp = df_census_pop_raw['GEOID'].str.slice(11,14).rename('County FIPS').astype(int)
# df_census_pop = pd.concat([df_census_pop_raw, state_fp, county_fp], axis=1).drop('GEOID', axis=1)
# df_census_pop

In [ ]:
# the re module provides regex matching operations
# import re 

# shapefile_tract = os.path.join(shapes_dir,"tiger2020PL_NY_tracts") # provide the full path to our shapefiles
# shapefile_tract_ny=[x for x in pathlib.Path(shapefile_tract).iterdir() if x.is_file()] # make a list of all the files in the directory with their full path

# tract_file_re = "tl_2020_[0-9]{5}" # use regex to find filenames containing a digit in the range 0-9, repeated 5 times 
# shapefile_tract_ny=[x for x in pathlib.Path(shapefile_tract).iterdir() if x.is_file() and re.search(tract_file_re, x.name)] # make a list of all files in the dir with their full path, as long as they meet above criteria

Given the [hierarchy through which census geographic entities are organized](https://www2.census.gov/geo/pdfs/reference/geodiagram.pdf), accessing New York City geographic data at the census tract level is not as straightforward as one might expect. Rather, we'll begin with the full set of NY state files, map population density, then draw a boundary around New York City. 

In [1]:
# myshapefile = os.path.join(shapes_dir, '/home/julie/git/portfolio/data/shapes/tiger2020PL_NY_tracts/tl_2020_36003_tract20.zip')
# mygeodf = gpd.read_file(myshapefile)

In [2]:
# df_state = pd.read_csv(file_state, 
#                        usecols=['STATE', 'STUSAB', 'STATE_NAME'], # use only these columns
#                        delimiter="|", # load txt file as pandas DataFrame 
#                        encoding="iso-8859-1", 
#                        encoding_errors='ignore')[['STATE_NAME', 'STUSAB', 'STATE',]] # reorder cols

In [3]:
# # search for any rows containing a comma in the "County" column
# df_place[df_place['County'].str.contains(',')]

# # srs_place_expanded = (df_place['County']
# #                           .str.strip( ) # strip any leading/trailing spaces
# #                           .str.split(",", expand=True) # split the string wherever a comma appears
# #                           .melt(ignore_index=False) # we want to preserve the index here, so we can use join to add it back to df_place
# #                           .dropna() # drop any rows with null values
# #                           .drop('variable', axis=1) # drop the new index column (labeled 'variable')
# #                           .rename(columns={'value': 'County'})
# #                          )

# # strip any leading/trailing spaces
# # split the string wherever a comma appears
# srs_place_expanded = (df_place['County'].str.strip( ).str.split(",", expand=True)) 

# # we want to preserve the index here, so we can use join to add it back to df_place
# # .dropna() # drop any rows with null values
# # .drop('variable', axis=1) # drop the new index column (labeled 'variable')
# # .rename(columns={'value': 'County'})
# srs_place_expanded = srs_place_expanded.melt(ignore_index=False).dropna().drop('variable', axis=1).rename(columns={'value': 'County'})
                          
# # rejoin our series into our df
# # adding rsuffix '_messy' as a reminder that this is the 'County' column we'll be dropping shortly
# df_place_expanded = srs_place_expanded.join(df_place, rsuffix='_messy')

# # now let's drop "County_messy"
# df_place_expanded = df_place_expanded.drop(['County_messy'], axis=1)

# # reorder the columns
# df_place_expanded = df_place_expanded[['State', 'State FIPS', 'Place', 'Place FIPS', 'County']]
# df_place_expanded

# # let's look at New York City to make sure the melt worked and each county really is in a new row 
# df_place_expanded.loc[df_place_expanded['Place'] == ("New York city")]
# # df_place_expanded.loc[df_place_expanded['Place'].str.startswith("New York")]

In [4]:
# df_codes = pd.merge(df_place_expanded, df_county, on=['State', 'State FIPS', 'County'], how = 'left') 
# df_codes =  df_codes[['State Name', 'State', 'State FIPS', 'Place', 'Place FIPS', 'County', 'County FIPS']] 
# df_codes.head(3)

In [5]:
# # need to strip whitepsace from df_place['County'] (apparently the melt thing above didn't do it...!)
# df_place_expanded['County'] = df_place_expanded['County'].str.strip()
# df_place_expanded.dtypes

In [6]:
# df_state['State FIPS'] = df_state['State FIPS'].astype(str)

In [7]:
geodf_tract_ny = pd.DataFrame() # create any empty DataFrame
for file in shapefile_tract_ny: # for every file in our list of shapefiles
    gpd.read_file(file) # read the file into a GEO DataFrame
    geodf_tract_ny = pd.concat([geodf_tract_ny, gpd.read_file(file)], ignore_index=True, copy=False) # concatenate each GEO DataFrame to geodf_tract_ny

geodf_tract_ny = geodf_tract_ny[['STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'GEOID20', 'ALAND20', 'geometry']]
geodf_tract_ny = geodf_tract_ny.rename(columns={'STATEFP20': 'State FIPS', 'COUNTYFP20': 'County FIPS', 'TRACTCE20': 'Census Tract Code', 'GEOID20': 'GEOID Census Tract', 'ALAND20': 'Land Area'}, inplace=True)
geodf_tract_ny = geodf_tract_ny.astype({'STATEFP20': 'int', 'COUNTYFP20':'int', 'TRACTCE20':'int', 'GEOID20': 'int', 'ALAND20': 'int'}).dtypes
geodf_tract_ny   

NameError: name 'pd' is not defined

# Create a DataFrame with a list of files from a folder
**...and split out the STATE FIPS and COUNTY FIPS codes from the filename.** We did this so that we have a df that has filenames of shapefiles along with the state FIPS and county FIPS they pertain to. We could use this to pre-filter the list of files we want to load using GeoPandas to only the shape files for a particular set of counties. For example, if we only want Bronx County - STATE FIPS 36 and COUNTY FIPS 57 - we could find the DF with these two numbers and only load the one file in that row.

In [ ]:
# this code block loops through our 'ny_shapefiles' list, and creates a separate list of FIPS codes for county and state 
county_codes=[] # create an empty list for County FIPS
state_codes=[] # create an empty list for State FIPS
filtered_shapefiles=[]

for file in ny_shapefiles: 
    filename_parts = file.name.replace(".zip","").split("_") # take each filename - remove '.zip', split the remaining string wherever "_"  appears, and save it as a list
    if len(filename_parts) >=3: # take every 'filename_parts' list containing 3 or more elements (fewer than 3 parts indicates a file is extraneous and we don't want it)
        if len(filename_parts[2]) ==5: # take from each list the element at index 2 (position 3), but only if it contains 5 digits [State FIPS + County FIPS = 5 digits]
            # filename_parts -->  tl_2020_36013_tract20.zip
            # 36013 <---filename_parts[2]
            # 013 <---filename_parts[2][1:4]
            county_codes.append(filename_parts[2][-3:]) # take the last 3 digits of the element at index 2, and append it to the list 
            state_codes.append(filename_parts[2][0:2])  # take the first 3 digits of the element at index 2, State FIPS, and append it to our list
            filtered_shapefiles.append(file)

# Using Pyarrow and Parquet on Shapefiles

In [ ]:
big_ny_shapefile = "../data/shapes/tiger2020PL_NY_tracts/tl_2020_36_tract20.zip"
small_clean_shapefile = "../data_archive/clean/tl_2020_36_tract20.parquet"

import geopandas as gpd
ny_shapes_df = gpd.read_file(big_ny_shapefile)[["GEOID20", "geometry"]]
#ny_shapes_df['STATEFP20'] = ny_shapes_df['STATEFP20'].astype(int)
#ny_shapes_df['COUNTYFP20'] = ny_shapes_df['COUNTYFP20'].astype(int)
#ny_shapes_df['TRACTCE20'] = ny_shapes_df['TRACTCE20'].astype(int)

import pyarrow as pa
import warnings; 
warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
ny_shapes_df.to_parquet(small_clean_shapefile, index=False, compression='BROTLI')
    #= pa.Table.from_pandas(ny_shapes_df, preserve_index=False)

In [ ]:
import numpy as np
import shapely.geometry.polygon as sp
import pandas as pd

def shrink_coords(polygon):
    if polygon.type != 'Polygon':
        return polygon
    x, y = polygon.exterior.coords.xy
    x = np.around(x, 5)
    y = np.around(y, 5)
    return sp.Polygon(list(zip(x,y)))

# Note: in this next line the simplify() function is  "supposed" to make a shape with 
# many points "simpler" (fewer points) / smaller
# Hwever,  in practice it either makes no difference or sometimes makes the file larger
#ny_shapes_df["geometry"].simplify(tolerance=5)

ny_shapes_df["geometry"] = ny_shapes_df["geometry"].apply(shrink_coords)

smaller_clean_shapefile = "../data_archive/clean/tl_2020_36_tract20_simp.parquet"
ny_shapes_df.to_parquet(smaller_clean_shapefile, index=False, compression='BROTLI')
ny_shapes_df

In [ ]:
type(ny_shapes_df["geometry"][0])

<!-- The Tally page tells us there are 3,143 Counties & Equivalents in the 50 states and DC. (This does not include Puerto Rico and the Island Areas.) We can check if the number of rows in our DF looks right by selecting all states and excluding PR (72), American Samoa (60), Guam (66), Commonwealth of the Northern Mariana Islands (69), United States Virgin Islands (78) and comparing this to the Tally page count (3,143). -->